In [1]:
import os
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
# URLs

# Transaction Fees
avg_transaction_fees = {'btc':'https://bitinfocharts.com/comparison/bitcoin-transactionfees.html',
                        'eth':'https://bitinfocharts.com/comparison/ethereum-transactionfees.html',
                        'bch':'https://bitinfocharts.com/comparison/bitcoin%20cash-transactionfees.html',
                        'ltc':'https://bitinfocharts.com/comparison/litecoin-transactionfees.html',
                        'xmr':'https://bitinfocharts.com/comparison/monero-transactionfees.html',
                        'dash':'https://bitinfocharts.com/comparison/dash-transactionfees.html',
                        'zec':'https://bitinfocharts.com/comparison/zcash-transactionfees.html'}

#median_transaction_fees = {'btc':'https://bitinfocharts.com/comparison/bitcoin-mediantransactionvalue.html',
#                           'eth':'https://bitinfocharts.com/comparison/ethereum-median_transaction_fee.html',
#                           'bch':'https://bitinfocharts.com/comparison/bitcoin%20cash-median_transaction_fee.html',
#                           'ltc':'https://bitinfocharts.com/comparison/litecoin-median_transaction_fee.html',
#                           'dash':'https://bitinfocharts.com/comparison/dash-mediantransactionvalue.html',
#                           }

# Block Times
block_times = {'btc': 'https://bitinfocharts.com/comparison/bitcoin-confirmationtime.html',
               'eth': 'https://bitinfocharts.com/comparison/ethereum-confirmationtime.html',
               'bch': 'https://bitinfocharts.com/comparison/bitcoin%20cash-confirmationtime.html',
               'ltc': 'https://bitinfocharts.com/comparison/litecoin-confirmationtime.html',
               'xmr': 'https://bitinfocharts.com/comparison/monero-confirmationtime.html',
               'dash': 'https://bitinfocharts.com/comparison/dash-confirmationtime.html',
               'zec': 'https://bitinfocharts.com/comparison/zcash-confirmationtime.html'}

In [10]:
def parse_record(record):
    date = record[11:21]
    value = record[24:-1]
    return [date,value]

dfs = []
for coin in avg_transaction_fees:
    url = avg_transaction_fees[coin]
    response = requests.get(url)
    soup = BeautifulSoup(response.text,'lxml')
    script_tag = soup.findAll('script')[5]
    script_text = script_tag.text


    pattern = re.compile(r'\[new Date\("\d{4}/\d{2}/\d{2}"\),[(\d)(\w)-.]*\]')
    records = pattern.findall(script_text)
    
    data = {'date':[],coin:[]}

    for record in records:
        parsed = parse_record(record)
        if parsed[1] == 'null':
            parsed[1] = None
        else:
            parsed[1] = float(parsed[1])
        data['date'].append(parsed[0])
        data[coin].append(parsed[1])
    
    df = pd.DataFrame(data).set_index('date')
    dfs.append(df)

In [11]:
avg_transaction_fee_df = pd.concat(dfs,axis=1)
avg_transaction_fee_df

,eth,dash,btc,bch,ltc,xmr,zec
2010/07/17,NaN,NaN,0.000010,NaN,NaN,NaN,NaN
2010/07/18,NaN,NaN,0.000243,NaN,NaN,NaN,NaN
2010/07/19,NaN,NaN,0.000022,NaN,NaN,NaN,NaN
2010/07/20,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010/07/21,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010/07/22,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010/07/23,NaN,NaN,0.000003,NaN,NaN,NaN,NaN
2010/07/24,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010/07/25,NaN,NaN,0.000097,NaN,NaN,NaN,NaN
2010/07/26,NaN,NaN,0.000357,NaN,NaN,NaN,NaN


In [12]:
dfs = []
for coin in block_times:
    url = block_times[coin]
    response = requests.get(url)
    soup = BeautifulSoup(response.text,'lxml')
    script_tag = soup.findAll('script')[5]
    script_text = script_tag.text


    pattern = re.compile(r'\[new Date\("\d{4}/\d{2}/\d{2}"\),[(\d)(\w)-.]*\]')
    records = pattern.findall(script_text)
    
    data = {'date':[],coin:[]}

    for record in records:
        parsed = parse_record(record)
        if parsed[1] == 'null':
            parsed[1] = None
        else:
            parsed[1] = float(parsed[1])
        data['date'].append(parsed[0])
        data[coin].append(parsed[1])
    
    df = pd.DataFrame(data).set_index('date')
    dfs.append(df)

In [13]:
block_times_df = pd.concat(dfs,axis=1)
block_times_df

,eth,dash,btc,bch,ltc,xmr,zec
2009/01/09,NaN,NaN,102.857,NaN,NaN,NaN,NaN
2009/01/10,NaN,NaN,23.607,NaN,NaN,NaN,NaN
2009/01/11,NaN,NaN,15.484,NaN,NaN,NaN,NaN
2009/01/12,NaN,NaN,15.319,NaN,NaN,NaN,NaN
2009/01/13,NaN,NaN,11.707,NaN,NaN,NaN,NaN
2009/01/14,NaN,NaN,11.163,NaN,NaN,NaN,NaN
2009/01/15,NaN,NaN,11.429,NaN,NaN,NaN,NaN
2009/01/16,NaN,NaN,13.333,NaN,NaN,NaN,NaN
2009/01/17,NaN,NaN,13.211,NaN,NaN,NaN,NaN
2009/01/18,NaN,NaN,13.458,NaN,NaN,NaN,NaN


In [14]:
avg_transaction_fee_df.to_csv('avg_transaction_fee.csv',index_label='date')
block_times_df.to_csv('block_times.csv',index_label='date')